In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import dlib
from dlib import rectangle


class FaceShapeModel(object):


    def __init__(self):
    
        model_filename = 'shape_predictor_68_face_landmarks.dat'
        if not os.path.exists(model_filename):
            os.system(
                'wget http://sourceforge.net/projects/dclib/files/dlib/v18.10/shape_predictor_68_face_landmarks.dat.bz2')
            os.system('bunzip2 shape_predictor_68_face_landmarks.dat.bz2')
        self.model = dlib.shape_predictor(model_filename)
        self.detector = dlib.get_frontal_face_detector()

    def localize(self, image):
       
        if image.dtype != np.uint8:
            image = image.astype(np.uint8)

        gray_img = None
        if image.ndim == 3:
            gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        elif image.ndim == 2:
            gray_img = image
        else:
            raise ValueError(
                'Unsupported number of image dimensions: %d' % image.ndim)

        faces = self.detector(gray_img)
        if len(faces):
            return [faces[0].left(), faces[0].top(), faces[0].right() - faces[0].left(), faces[0].bottom() - faces[0].top()]
        else:
            return None

    def fit(self, image):
      
        bbox = self.localize(image)
        if bbox is not None:
            face = rectangle(
                bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3])
            shape = self.model(image, face)
            xs = [p.x for p in shape.parts()]
            ys = [p.y for p in shape.parts()]
            x = np.min(xs)
            y = np.min(ys)
            w = np.max(xs) - np.min(xs)
            h = np.max(ys) - np.min(ys)
            return {
                'pts': zip(xs, ys),
                'xs': xs,
                'ys': ys,
                'pts-bbox': [x, y, w, h],
                'face-bbox': bbox,
                'nose': (shape.parts()[30].x, shape.parts()[30].y)
            }
        else:
            return None

    def draw(self, image, fr=None):
       
        if fr is None:
            fr = self.fit(image=image)

        plt.clf()
        plt.imshow(image)
        for pt in fr['pts']:
            plt.plot(pt[0], pt[1], 'ro')

